In [8]:
def bsm_mcs_valuation(strike):

    #
    # Monte Carlo valuation of European call option
    # in Black-Scholes-Merton model
    # bsm_mcs_euro.py
    #
    import numpy as np

    # Parameter Values
    S0 = 100.  # initial index level
#     K = 105.  # strike price
    T = 1.0  # time-to-maturity
    r = 0.05  # riskless short rate
    sigma = 0.2  # volatility

    I = 100000  # number of simulations

    # Valuation Algorithm
    z = np.random.standard_normal(I)  # pseudorandom numbers
    ST = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
      # index values at maturity
    hT = np.maximum(ST - K, 0)  # inner values at maturity
    C0 = np.exp(-r * T) * np.sum(hT) / I  # Monte Carlo estimator

    return C0
    # Result Output
#     print("Value of the European Call Option %5.3f" % C0)

In [9]:
def seq_value(n):
    
    strikes = np.linspace(80,120,n)
    option_values = []
    for strike in strikes:
        option_values.append(bsm_mcs_valuation(strike))
    return strikes, option_values

In [11]:
n = 1000
%time strikes, option_values_seq = seq_value(n)

CPU times: user 10 s, sys: 859 ms, total: 10.9 s
Wall time: 6.33 s


In [13]:
from IPython.parallel import Client
c = Client(profile='default')
view = c.load_balanced_view()

In [14]:
def par_value(n):
    strikes = np.linspace(80,120,n)
    option_values = []
    for strike in strikes:
        value = view.apply_async(bsm_mcs_valuation, strike)
        option_values.append(value)
    c.wait()
    return strikes, option_values

In [17]:
%time strikes, option_values_seq = par_value(n)

CPU times: user 10.5 s, sys: 711 ms, total: 11.2 s
Wall time: 13.9 s


In [ ]:
# ipython.parallel is slower